In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# list_data_name = ["BTCUSDT_1d", "ETHUSDT_1d", "BNBUSDT_1d", "XRPUSDT_1d", "ADAUSDT_1d", "LTCUSDT_1d"]                   
# list_data_name = ["BTCUSDT_1h", "ETHUSDT_1h", "BNBUSDT_1h", "XRPUSDT_1h", "ADAUSDT_1h", "LTCUSDT_1h"]                  

list_data_name = ["ADAUSDT_1h"]                  
                        
from_date = '2021-12-31 00:00:00' # 2020-12-31 00:00:00
until_date = '2025-07-31 00:00:00'
column_name = 'processed_log_return_wtmra_0'
slice_size_in_days = [1, 3, 7, 14, 21, 30]

In [3]:
def calculate_ROC(df, column_name_calculation, column_name_result, window_size=1):

    '''
    ROC = [(Close - Close n periods ago) / (Close n periods ago)] * 100
    ROC = Rate of Change
    '''
    
    df_copy = df.copy()
    
    df_copy[column_name_result] = df_copy[column_name_calculation].pct_change(periods=window_size)
    
    return df_copy


def calculate_derivative(df, column_name_calculation, column_name_result, window_size=1):
    
    df_copy = df.copy()
    
    df_copy[column_name_result] = df_copy[column_name_calculation].diff(periods=window_size)
    
    return df_copy

In [4]:
for data_name in list_data_name:
    # Importing data
    data = pd.read_csv(f'../data/01-output-{data_name}-from-{from_date}-until-{until_date}-log-return.csv', parse_dates=['date'], index_col='date')
    data = data[[column_name]]

    # for each slice size, create a lagged version of the data
    for slice_size in slice_size_in_days:
        data = calculate_ROC(data, column_name, f'{column_name}_ROC_{slice_size}_days', slice_size * 24)
        data = calculate_derivative(data, column_name, f'{column_name}_derivative_{slice_size}_days', slice_size * 24)    

    # remove the nan values
    data = data.dropna()

    # Exporting data
    data.to_csv(f'../data/02c-output-{data_name}-log-return-difference-and-change.csv')        